RAW (MOABB) to CSV

This code convert the data sets from RAW format to CSV format using MOABB.

It has been specifically conceived for BCI data.

This script is for bi2013a-AT 


In [ ]:
import numpy as np
import pandas as pd
import os
import shutil

In [ ]:
def reorganize(source_folder, destination_folder):
    """
    Reorganizes the 1.csv files from each subject/session into a new folder.
    Ensures subject and session numbers are formatted with leading zeros.

    Args:
        source_folder: Path to the folder containing all subjects
        destination_folder: Path to the folder where renamed files will be copied
    """
    # Create destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Loop through all subject folders
    for subject_folder in os.listdir(source_folder):
        subject_path = os.path.join(source_folder, subject_folder)

        # Check if it's a directory
        if not os.path.isdir(subject_path):
            continue

        # Extract subject number
        if subject_folder.startswith("subject"):
            if "_session" in subject_folder:
                # Format: subject01_session01
                subject_num = int(subject_folder.split("_session")[0].replace("subject", ""))
                session_num = int(subject_folder.split("_session")[1])
            else:
                # Format: subject08, subject09, etc.
                subject_num = int(subject_folder.replace("subject", ""))
                session_num = 1  # Only one session for subjects 8-24

            # Look for the Session folder
            for item in os.listdir(subject_path):
                if item.startswith("Session"):
                    session_path = os.path.join(subject_path, item)

                    # Look for the 1.csv file
                    csv_file = os.path.join(session_path, "1.csv")
                    if os.path.exists(csv_file):
                        # Create new filename
                        new_name = f"subject_{subject_num:02d}_session_{session_num:02d}.csv"
                        destination_path = os.path.join(destination_folder, new_name)

                        # Copy the file with the new name
                        shutil.copy2(csv_file, destination_path)
                        print(f"Copied: {csv_file} -> {destination_path}")

    print(f"Reorganization completed. Files are in: {destination_folder}")

In [ ]:
source = "C:\\Users\\doumif\\Downloads\\CSV zenodo bi2013a\\"
destination = "C:\\Users\\doumif\\Downloads\\CSV bi2013a-AT\\" 


In [ ]:
reorganize(source, destination)

In [ ]:
# Forward your file with all .csv of the dataset
# Cattan files have been downloaded from the zenodo repository, and then reorganized as you have seen above. We need to "clean" it before converting to npz
file_dir = "C:\\Users\\doumif\\Downloads\\CSV bi2013a-AT\\"
output_dir = "C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\P300\\bi2013a-AT"
all_files = [os.path.join(file_dir, file) for file in os.listdir(file_dir)]
all_files

In [ ]:
file = "C:\\Users\\doumif\\Downloads\\CSV bi2013a-AT\\subject_01_session_02.csv"

In [ ]:
df = pd.read_csv(file, header=0)
data = np.array(df)

In [ ]:
print(f"Unique values in stim column: {np.count_nonzero(data[:, 17])}\n")

In [ ]:
unique_values, counts = np.unique(data[:, 17], return_counts=True)
for value, count in zip(unique_values, counts):
    print(f"Dans data : Nombre de {value} = {count}")

In [ ]:
def process_eeg_file(file_path, output_dir):
    """
    Process a single EEG file and save it with a new format.

    Parameters:
    -----------
    file_path : str
        Path to the input CSV file
    output_dir : str
        Directory where processed file will be saved

    Returns:
    --------
    str
        Path to the processed file

    The function:
    - Reads the CSV file
    - Processes the data (merges target/non-target columns, removes unnecessary columns)
    - Saves processed file with format 'subject_XX_session_01.csv'
    """
    # Extract subject number from filename (assuming format like 'subject_01_PC.csv')
    filename = os.path.basename(file_path)

    # Read and process the data
    df = pd.read_csv(file_path, header=None)
    data = np.array(df)

    # 1. Merge target and non-target columns
    # Convert 33285 to 2 and 33286 to 1 in target column (column 18)
    data[:, 17] = np.where(data[:, 17] == 33285, 2, 
                  np.where(data[:, 17] == 33286, 1, 
                           data[:, 17]))

    # Convert back to DataFrame
    df_processed = pd.DataFrame(data)

    # Save processed file
    output_path = os.path.join(output_dir, filename)
    df_processed.to_csv(output_path, index=False, header=False)

    print(f"Processed and saved: {filename}")
    print(f"Shape: {data.shape}")
    unique_values, counts = np.unique(data[:, 17], return_counts=True)
    for value, count in zip(unique_values, counts):
        print(f"Dans data : Nombre de {value} = {count}")

    return output_path


In [ ]:
# 16 corresponds to the haeder
for i, file in enumerate(all_files, 1):
    print(f"File number {i} out of {len(all_files)}")
    csv = process_eeg_file(file, output_dir)
